In [1]:
import numpy as np

# Read the table data from the file
table_data = []
with open("table data.txt", "r") as file:
    lines = file.readlines()[1:]  # Skip the header line
    for line in lines:
        layer_data = line.strip().split()
        layer_number = int(layer_data[1])
        layer_type = layer_data[2]
        sdc_probability = float(layer_data[3])
        data_size = int(layer_data[4])
        total_extra_time = float(layer_data[5])
        table_data.append((layer_number, layer_type, sdc_probability, data_size, total_extra_time))

# Normalize the factors
max_sdc_probability = max(data[2] for data in table_data)
max_data_size = max(data[3] for data in table_data)
min_total_extra_time = min(data[4] for data in table_data)
max_total_extra_time = max(data[4] for data in table_data)

normalized_data = []
for layer_number, layer_type, sdc_probability, data_size, total_extra_time in table_data:
    normalized_sdc_probability = sdc_probability / max_sdc_probability
    normalized_data_size = data_size / max_data_size
    normalized_total_extra_time = (total_extra_time - min_total_extra_time) / (max_total_extra_time - min_total_extra_time) * 2 - 1
    
    # Assign computation type factor based on layer type
    if layer_type in ["Conv2d", "Linear"]:
        computation_type = 0.8  # Suitable for GPU execution
    else:
        computation_type = 0.2  # Suitable for TEE execution
    
    normalized_data.append((layer_number, normalized_sdc_probability, computation_type, normalized_data_size, normalized_total_extra_time))

# Define the objective function for parameter optimization
def objective_function(params):
    alpha, beta, gamma, delta = params
    total_costs = []
    for layer_number, sdc_probability, computation_type, data_size, total_extra_time in normalized_data:
        total_cost = alpha * sdc_probability + beta * computation_type + gamma * data_size + delta * total_extra_time
        total_costs.append((layer_number, total_cost))
    
    total_costs.sort(key=lambda x: x[1], reverse=True)
    top_11_layers = [layer[0] for layer in total_costs[:11]]
    
    partition_result = []
    with open("partition result.txt", "r") as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith("Label 'TEE':"):
                partition_result = [int(layer) for layer in line.split("[")[1].split("]")[0].split(", ")]
    
    score = sum(layer in top_11_layers for layer in partition_result)
    return -score  # Negative score for minimization

# Perform parameter optimization using grid search
alpha_values = np.linspace(0, 1, 11)
beta_values = np.linspace(0, 1, 11)
gamma_values = np.linspace(0, 1, 11)
delta_values = np.linspace(0, 1, 11)

best_params = None
best_score = float("inf")

for alpha in alpha_values:
    for beta in beta_values:
        for gamma in gamma_values:
            for delta in delta_values:
                params = (alpha, beta, gamma, delta)
                score = objective_function(params)
                if score < best_score:
                    best_params = params
                    best_score = score

print("Best parameters:")
print("Alpha (SDC Probability weight):", best_params[0])
print("Beta (Computation Type weight):", best_params[1])
print("Gamma (Data Size weight):", best_params[2])
print("Delta (Total Extra Time weight):", best_params[3])

Best parameters:
Alpha (SDC Probability weight): 0.1
Beta (Computation Type weight): 0.0
Gamma (Data Size weight): 0.0
Delta (Total Extra Time weight): 0.0


In [3]:
def assign_labels(params):
    alpha, beta, gamma, delta = params
    total_costs = []
    for layer_number, sdc_probability, computation_type, data_size, total_extra_time in normalized_data:
        total_cost = alpha * sdc_probability + beta * computation_type + gamma * data_size + delta * total_extra_time
        total_costs.append((layer_number, total_cost))
    
    total_costs.sort(key=lambda x: x[1], reverse=True)
    tee_layers = [layer[0] for layer in total_costs[:11]]
    gpu_layers = [layer[0] for layer in total_costs[11:]]
    
    return gpu_layers, tee_layers


test_params = (0.5, 0.5, 0.5, 0.5)  # Test parameters for demonstration

# Assign labels to layers using the best parameters
gpu_layers, tee_layers = assign_labels(best_params)

print("Layers assigned to GPU:", sorted(gpu_layers))
print("Layers assigned to TEE:", sorted(tee_layers))

Layers assigned to GPU: [1, 3, 4, 6, 7, 8, 10, 11, 13, 14, 15, 17, 18, 20, 21, 23, 24, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60]
Layers assigned to TEE: [2, 5, 9, 12, 16, 19, 22, 25, 29, 38, 54]
